In [ ]:
import matplotlib.pyplot as plt
import pickle
import numpy as np

method = "LN_COBYLA (NLopt)"
problem = "maxcut"
depth_pool = [5]
qubit_pool = list(range(10, 21, 2))
n = 10
seed_pool = list(range(100))

for p in depth_pool:
    # for i, n in enumerate(qubit_pool):
    data = pickle.load(
        open(
            f"data/{problem}/configs/rhobeg/{method}-p{p}-q{n}-s{seed_pool[0]}-{seed_pool[-1]}.pckl",
            "rb",
        ),
    )
    config = data["config"]
    result = data["result"]
    maxfev_pool = config["maxeval"]
    rhobeg_pool = config["initial_step"]
    result = result.reshape(-1, len(seed_pool), len(maxfev_pool), len(rhobeg_pool))
    # result = result.reshape(len(qubit_pool), len(seed_pool), *result.shape[1:])
    initial_ar = data["initial_ar"]
    print(config)

    plt.figure(figsize=(12, 4))
    bar_width = 0.4
    colors = ["tab:blue", "tab:orange", "pink", "lightblue", "lightgreen", "violet"]
    for i, (maxfev, c) in enumerate(zip(maxfev_pool, colors)):
        ys = np.array(rhobeg_pool)[np.argmax(result[:,:,i], axis=2)]
        print(np.median(ys[0]), np.mean(ys[0]))
        plt.violinplot(
            ys.T,
            # sym="+",
            positions=np.array(qubit_pool)+bar_width*i,
            widths=bar_width,
            # showmeans=True,
            showmedians=True,
            # boxprops=dict(facecolor=c, alpha=0.75),
            # flierprops=dict(color=c, markeredgecolor=c, markerfacecolor=c),
            # medianprops=dict(color="yellow"),
            # patch_artist=True,
        )
        plt.plot([], c=c, label=f"{maxfev=}")
    plt.xticks(np.array(qubit_pool) + bar_width * (len(maxfev_pool)-1) / 2, labels=qubit_pool)
    plt.ylabel("rhobeg")
    plt.title(f"{p=}")
    plt.legend()

    plt.figure()
    for i, (maxfev, c) in enumerate(zip(maxfev_pool, colors)):
        ys = result[0,:,i].reshape(-1, len(rhobeg_pool))
        plt.errorbar(rhobeg_pool, np.mean(ys, axis=0), yerr=np.std(ys, axis=0), c=c, label=f"{maxfev=}")
    # plt.ylim((0.9, 1))
    plt.ylabel("AR")
    plt.xlabel("rhobeg")
    plt.legend()
